Imports:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn import linear_model as lm


In [ ]:
Data Retrieval

In [ ]:
dialogue = pd.read_csv(("https://raw.githubusercontent.com/"
 "rfordatascience/tidytuesday/master/data/"
 "2020/2020-09-08/friends.csv"))

episodes = pd.read_csv(("https://raw.githubusercontent.com/"
 "rfordatascience/tidytuesday/master/data/"
 "2020/2020-09-08/friends_info.csv"))

print("Example Dialogue:")
dialogue.head(5)

print("Example Episode:")
episodes.head(5)